### Outcome number of days sick

In [1]:
### 2005 measures of reciprocity
### 2006 controls
### 2007 days off work in 2006

import pandas as pd
import numpy as np
import seaborn as sns
import math

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols



# define path: insert the path where the SOEP data is stored on your computer here
from pathlib import Path
## copy your path in here:
# Path Max: C:/Users/max-admin/Desktop/Masterstudium/WiSe_22_23/Research_Module/SOEP-Data/Stata/raw
# Path Maxie: /Volumes/dohmen_soep/SOEP-CORE.v36eu_STATA/Stata/raw

data_folder = Path("C:/Users/max-admin/Desktop/Masterstudium/WiSe_22_23/Research_Module/SOEP-Data/Stata/raw")
# define relevant subsets of SOEP-data
file_names = ['vp', 'wp', 'xp']

file_paths = [data_folder / f"{file_name}.dta" for file_name in file_names]
# some controls are in gen data
file_paths_2 = [data_folder / f"{file_name}gen.dta" for file_name in file_names]

In [2]:
# read in 2005 data for the reciprocity measures
data05 = pd.read_stata(file_paths[0], columns=["pid","hid","vp12602", "vp12603", "vp12605"]).set_index(['pid', 'hid'])
df_2005 = data05.rename(columns={ 'vp12602': 'take_revenge', 'vp12603': 'similar_problems', 'vp12605': 'insult_back'})
# create dummies for take_revenge question
# Create dummy variables
dummies = pd.get_dummies(df_2005['take_revenge'])

# Join the dummy variables to the original dataframe
df_2005 = pd.concat([df_2005, dummies], axis=1)
# rename dummy
df_05 = df_2005.rename(columns = {'[1] Trifft ueberhaupt nicht zu' : 'rec1' , '[2] Skala 1-7' : 'rec2' , '[3] Skala 1-7' : 'rec3' ,'[4] Skala 1-7' : 'rec4' ,'[5] Skala 1-7' : 'rec5' ,'[6] Skala 1-7' : 'rec6' ,'[7] Trifft voll zu' : 'rec7'})
df_05 = df_05.drop(columns = ["[-1] keine Angabe"])

In [23]:
# read in 2006 data
# personal = personal advancement
# still includes all unfair treat
data06 = pd.read_stata(file_paths[1], columns=["pid", "hid", 'wp43b01', 'wp43b02', 'wp43b03', 'wp43b04', 'wp43b05', 'wp43b06', 'wp43b07','wp43b08' ]).set_index(['pid', 'hid'])
df_06 = data06.rename(columns={ 'wp43b01': 'recog_sup', 'wp43b02': 'felt_recog_sup',"wp43b03": "recog_effort",  'wp43b04': 'felt_recog_effort', "wp43b05": "recog_personal", "wp43b06" :"felt_recog_personal" ,"wp43b07": "recog_pay",'wp43b08': 'felt_recog_pay'})


## this next one one has to include ALL of the following columns: done
data2 = pd.read_stata(file_paths[1], columns=["pid", "hid","wp12402", "wp12401", "wp5802",  "wp5902", "wp4101", "wp17", "wp1901", "wp0102" ]).set_index(['pid', 'hid'])
df_06insteadof07 = data2.rename(columns={"wp12401": "gender", "wp12402": "year_birth", "wp5902": "wage_lastmonth", "wp4101": "commute_distance", 'wp5802': 'overtime', 'wp17': 'new_job', 'wp1901': 'reason_new_job' , "wp0102" : "work_satisfaction"})

# change has to include all of the following
hours06 = pd.read_stata(file_paths_2[1], columns=["pid","hid","wvebzeit", "betr06", "wpsbil", "werwzeit", "wbilzeit"]).set_index(['pid', 'hid'])
df_work06insteadof07 = hours06.rename(columns={'wvebzeit': 'working_hours', "betr06": "firmsize", "wpsbil": "school_degree", "werwzeit": "tenure" , "wbilzeit" : "years_educ"})

absent_days =  pd.read_stata(file_paths[2], columns=["pid", "hid",'xp10402','xp103']).set_index(['pid', 'hid'])
absent_days.rename(columns= {"xp10402" : "days_absent" , "xp103" : "long"}, inplace = True)


long_term = absent_days[(absent_days["long"] == '[1] Einmal') | (absent_days["long"] == '[2] Mehrmals' )].index
absent_days.drop(index=long_term, inplace = True)
absent_days.drop(index=absent_days[absent_days['days_absent'] < 0].index, inplace=True)



absent_days

,,days_absent,long
pid,hid,,
1704,175,30,[3] Nein
5203,125466,21,[3] Nein
5303,105600,5,[3] Nein
6002,604,30,[3] Nein
8606,127426,15,[3] Nein
...,...,...,...
8267002,826707,2,[3] Nein
8267502,826758,6,[3] Nein
8268502,826855,5,[3] Nein


In [54]:
# mapping for reciprocity questions: same scale for all
reciprocity_questions_mapping = {
    '[1] Trifft ueberhaupt nicht zu': 1,
    '[2] Skala 1-7': 2,
    '[3] Skala 1-7': 3,
    '[4] Skala 1-7': 4,
    '[5] Skala 1-7': 5,
    '[6] Skala 1-7': 6,
    '[7] Trifft voll zu': 7,
    '[-1] keine Angabe': -1,
}
## mapping for recognition questions: binary -> binary -> unfair treatment: No -> later 1 fair treatment: Yes -> 0
recog_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
    '[1] Ja': 2,
    '[2] Nein': 1,
}
# felt recog mapping
felt_recog_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
    '[1] Gar nicht': 1,
    '[2] Maessig': 2,
    '[3] Stark': 3,
    '[4] Sehr stark': 4,
}
# mapping for firmsize -> we need to recode this in a sensible way: jumps are the same: first change: selbstständig to 0
firmsize_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
    '[1] Unter  5': 1,
    '[2] 5 bis 10': 2,
    '[3] 11 bis unter 20': 3,
    '[4] bis 90: unter 20': 4,
    '[5] 91-04: 5 bis unter 20': 5,
    '[6] 20 bis unter 100': 6,
    '[7] 100 bis unter 200': 7,
    '[8] bis 98: 20 bis unter 200': 8,
    '[9] 200 bis unter 2000': 9,
    '[10] 2000 und mehr': 10,
    '[11] Selbstaendig-ohne Mitarb.': 0,
}
# mapping new job into binary variable
new_job_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1, 
    '[1] Ja': 1, 
    '[2] Nein': 2,
    '[3] Ja, nach Datenpruefung': 1,
}
# mapping for turnover intention: split up into binary with roughly equal value counts for simplicity: might change that later to categories
turnover_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe':-1,
    '[0] 0% wahrscheinlich': 2,
    '[10] 10% wahrscheinlich': 1,
    '[20] 20% wahrscheinlich': 1,
    '[30] 30% wahrscheinlich': 1,
    '[40] 40% wahrscheinlich': 1, 
    '[50] 50% wahrscheinlich': 1,
    '[60] 60% wahrscheinlich': 1,
    '[70] 70% wahrscheinlich': 1,
    '[80] 80% wahrscheinlich': 1,
    '[90] 90% wahrscheinlich': 1,
    '[100] 100% wahrscheinlich': 1,
}
# mapping for turnover intention ROBUSTNESS CHECK: Cardinal
turnover_mapping_cardinal = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe':-1,
    '[0] 0% wahrscheinlich': 0,
    '[10] 10% wahrscheinlich': 10,
    '[20] 20% wahrscheinlich': 20,
    '[30] 30% wahrscheinlich': 30,
    '[40] 40% wahrscheinlich': 40, 
    '[50] 50% wahrscheinlich': 50,
    '[60] 60% wahrscheinlich': 60,
    '[70] 70% wahrscheinlich': 70,
    '[80] 80% wahrscheinlich': 80,
    '[90] 90% wahrscheinlich': 90,
    '[100] 100% wahrscheinlich': 100,
}
# work satisfaction mapping
satisfaction_mapping = {
    '[0] 0 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[1] 1 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[2] 2 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[3] 3 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[4] 4 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[5] 5 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[6] 6 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 1,
    '[7] 7 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 1,
    '[8] 8 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 1,
    '[9] 9 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 1,
    '[10] 10 Zufrieden: Skala 0-Niedrig bis 10-Hoc': 1,
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
}
# mapping for new job to easier remove negatives
reason_new_job_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe':-1, 
    '[1] Erstmals erwerbstaetig': 1,
    '[2] Wieder erwerbstaetig': 2,
    '[3] Stelle bei neuen Arbeitgeber': 3,  
    '[4] Uerbnommen von Betrieb': 4,
    '[5] Stellenwechsel im Betrieb': 5, 
    '[6] Selbstaendig geworden': 6,
}
# mapping for school degree: to easier remove negatives
school_degree_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe':-1,
    '[1] Hauptschulabschluss': 1,
    '[2] Realschulabschluss': 2,
    '[3] Fachhochschulreife': 3,
    '[4] Abitur': 4,
    '[5] Anderer Abschluss': 5,
    '[6] Ohne Abschluss verlassen': 6,
    '[7] Noch kein Abschluss': 7,
    '[8] Keine Schule besucht': 8,
}
# reversed mapping to redo changes
reversed_mapping_reason = {v: k for k, v in reason_new_job_mapping.items()}
reversed_mapping_schoold = {v: k for k, v in school_degree_mapping.items()}
# mapping for binary reciprocity measure: ROBUSTNESS CHECK
def binary_reciprocity(x):
    if x >= 3:
        return 1 # High Group
    else:
        return 0 # Low Group

def recode_categoricals(inputdf):
    """
        recodes categoricals according to defined mappings

        Input:
            - inputdf : merged dataframe
            - rc_cardinal: optional argument: if == 1 turnover intentions will be coded as cardinal instead of binary 
            - rc_rec_binary: optional argument: if == 1 neg. reciprocity will be split into 2 groups: high and low
        Output:
            - df : cleaned Dataframe

    """
    merged = inputdf  
    
    # recode Gender variable
    merged['gender'].replace('[2] Weiblich', 2,inplace=True)
    merged['gender'].replace('[1] Maennlich', 1,inplace=True)
    # recode reciprocity variables
    merged[["similar_problems","take_revenge","insult_back"]] = merged[["similar_problems","take_revenge","insult_back"]].apply(lambda x: x.map(reciprocity_questions_mapping))
    # recode recognition variables
    merged[["recog_sup","recog_effort","recog_personal","recog_pay"]] = merged[["recog_sup","recog_effort","recog_personal","recog_pay"]].apply(lambda x: x.map(recog_mapping))
    # recode felt recognition variables
    merged[["felt_recog_sup","felt_recog_effort","felt_recog_personal","felt_recog_pay"]] = merged[["felt_recog_sup","felt_recog_effort","felt_recog_personal","felt_recog_pay"]].apply(lambda x: x.map(felt_recog_mapping))
    # recode firm size
    merged['firmsize'] = merged['firmsize'].map(firmsize_mapping)
    # recode work-satisfaction
    merged['work_satisfaction'] = merged['work_satisfaction'].map(satisfaction_mapping)
    # recode new job reason variable
    merged['reason_new_job'] = merged['reason_new_job'].map(reason_new_job_mapping)
    # recode job change variable
    merged['new_job']= merged['new_job'].map(new_job_mapping)
    # recode turnover intention variable

    # recode school degree
    merged['school_degree'] = merged['school_degree'].map(school_degree_mapping)
    
    output = merged
    return output

#def merge_and_clean(df_05,df_06,df_07,work07,rc_cardinal = 0,rc_rec_binary=0):
def merge_and_clean(df_05,df_06,df_07,work07,rc_cardinal = 0,rc_rec_binary=0):
    """
    merges data from different years, applies recoding to categoricals and constructs additional variables.

    Input:
        - df_05 : Pd.Dataframe contains reciprocity measures
        - df_06 : Pd.Dataframe contains unfair treatment measures
        - df_07 : Pd.Dataframe contains outcome and controls
        - work07 : Pd.Dataframe contains additional controls

    Output:
        - df : cleaned Dataframe

    """
    allmerged_df = pd.merge(pd.merge(pd.merge(df_05,df_06,on=["pid", "hid"]),df_07,on=["pid","hid"]),work07,on=["pid", "hid"])
    recoded = recode_categoricals(allmerged_df).astype(float)

    # replaces negative values with n.a.n 
    recoded = recoded.mask(recoded < 0, np.nan)
    recoded["recog_effort"].fillna(value= -0.5, inplace =True)
    # construct avg reciprocity measure
    recoded['avg_rec'] = recoded[['take_revenge', 'similar_problems', 'insult_back']].mean(axis=1)
    # For robustness check: option to construct binary reciprocity measure.
    if rc_rec_binary == 1:
        recoded['binary_rec'] = recoded['avg_rec'].apply(binary_reciprocity)
    
    # construct age, potential experience and age^2
    recoded['age'] = 2007 - recoded['year_birth']
    recoded["potential_experience"] = pow((recoded["age"] - 18), 2)
    recoded["age_squared"] = (recoded["age"] ** 2) / 100
    # recode categoricals back to make it better readable
    recoded["reason_new_job"] = recoded["reason_new_job"].map(reversed_mapping_reason)
    recoded["school_degree"] = recoded["school_degree"].map(reversed_mapping_schoold)

    # transform binary variables with 1 and 2 into 1 and 0
    columns_to_transform = ["recog_sup","recog_effort", "recog_pay", "recog_personal" ,"gender", "new_job"]

    # Iterate over the columns and replace the values 2 with 0 
    for col in columns_to_transform:
        recoded[col] = recoded[col].replace({2: 0})

    # save df somewhere so its not muted when repeatedly executing this cell: Can later transform that into functions
    df = recoded
    
    return df

# Add Mincer Wage Regression and adds its residuals to the Dataframe

def add_mincer_residuals(cleaneddata):
    
    df_cleaned = cleaneddata
    # specify which columns to drop from our dataframe
    df_mincer = cleaneddata.drop(columns=['similar_problems', 'take_revenge', 'insult_back', 'felt_recog_sup', 'felt_recog_effort',
       'recog_personal', 'felt_recog_personal', 'recog_pay', 'felt_recog_pay', 'year_birth', 'new_job',
       'reason_new_job', 'school_degree','overtime', 'recog_sup', 'age', 'commute_distance', 'recog_effort', 'working_hours','work_satisfaction','rec1' , 'rec2' , 'rec3' , 'rec4', 'rec5' , 'rec6' , 'rec7'])
    # Convert 'gender' and 'sector' columns to categorical data type
    df_mincer['gender'] = df_mincer['gender'].astype('category')
    df_mincer = df_mincer.dropna()
    # Define the dependent variable
    y = df_mincer['wage_lastmonth']
    # Define the independent variables
    X = df_mincer[['gender', 'firmsize', 'tenure', 'years_educ', 'potential_experience', 'age_squared']]

    # Add a constant term to the independent variables
    X = sm.add_constant(X)

    # Fit the Mincer wage regression model
    mincer_model = sm.OLS(y, X).fit()
    # Create a new column in the dataframe with the same name as the residuals array/ delete relative wage entries
    df_cleaned['mincer_residuals'] = None
    # Match the rows of the dataframe with the values in the residuals array using the index
    df_cleaned.loc[df_cleaned.index, 'mincer_residuals'] = mincer_model.resid

    return df_cleaned

In [51]:
data_2006 = add_mincer_residuals(merge_and_clean(df_05,df_06,df_06insteadof07,df_work06insteadof07))
# drop n.a.ns.
data_absenteism = pd.merge(data_2006, absent_days, on = ["pid","hid"])
data_absenteism


,,take_revenge,similar_problems,insult_back,rec1,rec2,rec3,rec4,rec5,rec6,rec7,...,school_degree,tenure,years_educ,avg_rec,age,potential_experience,age_squared,mincer_residuals,days_absent,long
pid,hid,,,,,,,,,,,,,,,,,,,,,
1704,175,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,[2] Realschulabschluss,16.833,12.0,1.000000,51.0,1089.0,26.01,905.154345,30,[3] Nein
5203,125466,1.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,2.333,NaN,1.333333,25.0,49.0,6.25,NaN,21,[3] Nein
5303,105600,5.0,4.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,[2] Realschulabschluss,7.250,11.5,4.333333,38.0,400.0,14.44,352.803178,5,[3] Nein
6002,604,3.0,3.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,[1] Hauptschulabschluss,NaN,10.5,3.000000,57.0,1521.0,32.49,NaN,30,[3] Nein
9201,922,3.0,2.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,[4] Abitur,6.750,15.0,2.333333,53.0,1225.0,28.09,-198.977863,14,[3] Nein
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8250802,825085,3.0,2.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,[4] Abitur,0.917,13.0,2.333333,24.0,36.0,5.76,-1193.060000,4,[3] Nein
8256802,825689,3.0,3.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,[2] Realschulabschluss,3.583,11.5,3.666667,25.0,49.0,6.25,388.303103,8,[3] Nein
8261002,826103,5.0,4.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,[2] Realschulabschluss,1.417,10.0,4.333333,22.0,16.0,4.84,NaN,8,[3] Nein


In [52]:

data_absenteism.drop(columns=['similar_problems', 'take_revenge', 'insult_back', 'felt_recog_sup', 'felt_recog_effort',
       'recog_personal', 'felt_recog_personal', 'recog_pay', 'felt_recog_pay', 'year_birth', 'new_job',
       'reason_new_job', 'school_degree','overtime', 'recog_sup','wage_lastmonth', 'rec1' , 'rec2' , 'rec3' , 'rec4', 'rec5' , 'rec6' , 'rec7' , 'work_satisfaction', 'long' ], inplace = True)
#  included 'recog_effort'again , excluded overtime due to sample size and wage last month
# add interaction term
data_absenteism["recXrecog_effort"] = data_absenteism["recog_effort"] * data_absenteism["avg_rec"]
# drop missing data in regression dataframe
data_absenteism.dropna(inplace=True)
data_absenteism

,,recog_effort,gender,commute_distance,working_hours,firmsize,tenure,years_educ,avg_rec,age,potential_experience,age_squared,mincer_residuals,days_absent,recXrecog_effort
pid,hid,,,,,,,,,,,,,,
1704,175,-0.5,0.0,10.0,41.000000,10.0,16.833,12.0,1.000000,51.0,1089.0,26.01,905.154345,30,-0.500000
5303,105600,-0.5,1.0,4.0,40.000000,7.0,7.250,11.5,4.333333,38.0,400.0,14.44,352.803178,5,-2.166667
9302,930,0.5,0.0,5.0,28.000000,6.0,17.500,18.0,2.666667,44.0,676.0,19.36,222.500374,2,1.333333
9401,949,-0.5,0.0,6.0,19.299999,10.0,1.083,13.0,4.333333,47.0,841.0,22.09,-574.827987,8,-2.166667
9402,949,-0.5,1.0,4.0,36.500000,10.0,20.083,12.0,3.000000,44.0,676.0,19.36,202.542481,10,-1.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8246405,824640,-0.5,1.0,4.0,39.500000,9.0,2.417,14.5,2.000000,24.0,36.0,5.76,-142.617201,3,-1.000000
8249802,824984,0.5,0.0,5.0,32.000000,6.0,13.583,18.0,4.000000,48.0,900.0,23.04,139.416805,2,2.000000
8250402,825042,-0.5,0.0,25.0,38.500000,10.0,3.333,11.5,4.333333,28.0,100.0,7.84,632.307755,5,-2.166667


In [53]:
# Define Outcome variable
Y = data_absenteism["days_absent"]
# define X matrix
X = data_absenteism.drop(columns=["days_absent"])
# add constant 
X = sm.add_constant(X)
from statsmodels.regression.linear_model import OLS
model = OLS(Y, X.astype(float))
lpm_model = model.fit(cov_type= "HC3")
lpm_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            days_absent   R-squared:                       0.054
Model:                            OLS   Adj. R-squared:                  0.051
Method:                 Least Squares   F-statistic:                     264.5
Date:                Thu, 12 Jan 2023   Prob (F-statistic):               0.00
Time:                        00:38:11   Log-Likelihood:                -11021.
No. Observations:                3012   AIC:                         2.207e+04
Df Residuals:                    2999   BIC:                         2.215e+04
Df Model:                          12                                         
Covariance Type:                  HC3                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    1.6866      0.223      7.570      0.000       1.250       2.123
recog_effort             3.0854      0.920      3.354      0.001       1.282       4.888
gender                  -1.7421      0.397     -4.390      0.000      -2.520      -0.964
commute_distance         0.0071      0.003      2.030      0.042       0.000       0.014
working_hours            0.0741      0.026      2.866      0.004       0.023       0.125
firmsize                 0.2048      0.064      3.190      0.001       0.079       0.331
tenure                  -0.0048      0.024     -0.198      0.843      -0.052       0.043
years_educ              -0.6427      0.061    -10.569      0.000      -0.762      -0.524
avg_rec                 -0.0548      0.138     -0.398      0.691      -0.325       0.215
age                      1.6156      0.184      8.797      0.000       1.256       1.976
potential_experience     0.0536      0.008      6.948      0.000       0.038       0.069
age_squared             -4.8823      0.657     -7.432      0.000      -6.170      -3.595
mincer_residuals        -0.0013      0.000     -5.176      0.000      -0.002      -0.001
recXrecog_effort        -0.6406      0.264     -2.425      0.015      -1.158      -0.123
==============================================================================
Omnibus:                     2084.351   Durbin-Watson:                   1.940
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            42716.832
Skew:                           3.024   Prob(JB):                         0.00
Kurtosis:                      20.430   Cond. No.                     7.29e+17
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
[2] The smallest eigenvalue is 4.65e-27. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""